In [0]:
import os
import requests
from urllib.parse import urljoin

try:
  import goodtables
except:
  !pip install goodtables
  import goodtables
try:
  import tableschema
except:
  !pip install tableschema
  import tableschema
try:
  import tableschema_pandas
except:
  !pip install tableschema_pandas
  import tableschema_pandas
try:
  import graphviz
except:
  !pip install graphviz
  !apt-get install graphviz
  import graphviz
from tableschema import Schema
#!apt-get install graphviz
# NOTE: must restart kernal after doing this

try:
  import statsmodels
except:
  !pip install statsmodels
  import statsmodels


# Validate your data against data schemas

 1. find and validate schemas themselves
 2. find and validate data

Clean your JSON with https://jsonlint.com/
Check against schemas with https://try.goodtables.io/

In [0]:
GITHUB_USER  = "e-lo"
GITHUB_REPO  = "forecast-cards"
BRANCH       = "master"
SHA          = "f35185168b238429157adcbf5ba09d09ae7d0172"
SUBDIRS      = ["examples"] #specify at least one
repo_api = "".join(["https://api.github.com/repos/",GITHUB_USER,"/",GITHUB_REPO,"/git/trees/",SHA,"?recursive=1"])
repo_raw = "".join(["https://raw.github.com/",GITHUB_USER,"/",GITHUB_REPO,"/",BRANCH,"/"])

## Find and validate **schema**s

In [0]:
## specify which schemas to use
schemas_loc = {
    "poi": urljoin(repo_raw,"poi-schema.json"),
    "scenario": urljoin(repo_raw,"scenario-schema.json"),
    "project": urljoin(repo_raw,"project-schema.json"),
    "observations": urljoin(repo_raw,"observations-schema.json"),
    "forecast": urljoin(repo_raw,"forecast-schema.json"),
}

## Confirm Schemas are valid
schemas = {}
for k,v in schemas_loc.items():
  print ("Obtaining ", k,"schema from: ",v)
  schemas[k] = Schema(v)
#requests.get(v)
#print(schemas['poi'])

In [0]:
from graphviz import Digraph
erd_graph = Digraph(name='Schemas', node_attr={'shape': 'plain'})


for k,v in schemas.items():
  node_label="<<table border='0' cellborder='1' cellspacing='0'>"
  node_label+="<tr><td><b>"
  node_label+=str(k)
  node_label+="</b></td></tr>"
  for f in v.descriptor['fields']:
    node_label+="<tr><td port='"
    node_label+=f['name']
    node_label+="'>"
    node_label+=f['name']
    node_label+="</td></tr>"
  node_label+="</table>>"
  erd_graph.node(k,label = node_label)

erd_graph.edge("poi:poi_id", "observations:poi_id")
erd_graph.edge("observations:forecast_id","forecast:forecast_id")
erd_graph.edge("scenario:run_id","forecast:run_id")
erd_graph.edge("project:project_id","scenario:project_id")

erd_graph 
  

## Locate Data

In [0]:
r = requests.get(repo_api)
rj = r.json()

In [0]:
card_locs = {
           "poi": [],
           "scenario": [],
           "project": [],
           "observations": [],
           "forecast": [],
    }

## todo better regex matching by project
for file in rj['tree']:
  path_list = file['path'].split("/")
  
  if len(path_list)>2 and path_list[0] in SUBDIRS and file['type']=='blob':
      if path_list[-1][0:8].lower()=="forecast":
          full_url = urljoin(repo_raw,file['path'])
          print("adding",full_url,"to forecast")
          card_locs["forecast"].append(full_url)
      if path_list[-1][0:12].lower()=="observations":
          full_url = urljoin(repo_raw,file['path'])
          print("adding",full_url,"to observations")
          card_locs["observations"].append(full_url)
      if path_list[-1][0:3].lower()=="poi":
          full_url = urljoin(repo_raw,file['path'])
          print("adding",full_url,"to poi")
          card_locs["poi"].append(full_url)          
      if path_list[-1][0:7].lower()=="project":
          full_url = urljoin(repo_raw,file['path'])
          print("adding",full_url,"to project")
          card_locs["project"].append(full_url)     
      if path_list[-1][0:8].lower()=="scenario":
          full_url = urljoin(repo_raw,file['path'])
          print("adding",full_url,"to scenario")
          card_locs["scenario"].append(full_url)   
                    
#card_locs


## Validate Data Tables

If errors found, try https://try.goodtables.io as a good GUI for identifying issues.

In [0]:
from goodtables import validate
reports={}
for k,v in card_locs.items():
  #print ("validating",k,v)
  reports[k] = validate(card_locs[k][0],schema=requests.get(schemas_loc[k]).json())
  if not reports[k]['valid']: 
    print ("--->INVALID TABLE", k)
    reports[k]
  else:
    print ("--->VALID",k)

In [0]:
reports['scenario']

# Create Estimation File

1. Merge tables on keys
2. Clean unusable records based on required variables
3. Create categorical variables and then dummy variables
4. Scale variables by forecast_value


## Merge tables


In [0]:
import pandas as pd
project_df = pd.concat([pd.read_csv(f, parse_dates=['year_open_planned','year_horizon','date_open_actual'],infer_datetime_format=True) for f in card_locs["project"]], ignore_index=True)
scenario_df = pd.concat([pd.read_csv(f, parse_dates=['forecast_creation_date','scenario_date'],infer_datetime_format=True) for f in card_locs["scenario"]], ignore_index=True)
scenario_proj_df = scenario_df.merge(project_df, on='project_id', how='left')

poi_df = pd.concat([pd.read_csv(f) for f in card_locs["poi"]], ignore_index=True)
observations_df = pd.concat([pd.read_csv(f, dtype={'obs_value':float}) for f in card_locs["observations"]], ignore_index=True)
observations_poi_df = observations_df.merge(poi_df, on='poi_id', how='left')

forecast_df = pd.concat([pd.read_csv(f, dtype={'forecast_value':float}) for f in card_locs["forecast"]], ignore_index=True)

all_df = forecast_df.merge(observations_poi_df, on='forecast_match_id', how='left').merge(scenario_proj_df, on='run_id', how='left')


In [0]:
all_df.dtypes

## Clean Dataset

Only keep records that don't have NULL for required variables

In [0]:
REQUIRED_VARS = ['scenario_date','forecast_system_type','forecast_creation_date','forecast_value','forecast_value','obs_value', 'area_type', 'facility_type','state','project_type']
usable_df= all_df.dropna(subset=REQUIRED_VARS)
print("Kept",len(usable_df),"of",len(all_df))


## Create categorical variables

1. Decade for forecast creation and scenario
2. Large projects

In [0]:
## categorical decades variable
usable_df['creation_decade'] = (usable_df['forecast_creation_date'].apply(lambda x: x.year//10*10)).astype('category')
usable_df['scenario_decade'] = (usable_df['scenario_date'].apply(lambda x: x.year//10*10)).astype('category')

## large projects dummy variable
breakpoint = 30000
bins = [usable_df['forecast_value'].min(), breakpoint, breakpoint+usable_df['forecast_value'].max()]
labels = ["small_project","large_project"]
usable_df['project_size'] = pd.cut(usable_df['forecast_value'], bins=bins, labels=labels)
#usable_df

## Turn categorical variables into dummy variables

In [0]:
categorical_cols = ['project_size','creation_decade','scenario_decade','functional_class','forecast_system_type','project_type','agency','forecaster_type','area_type','facility_type','state']
dummy_df = pd.get_dummies(usable_df[categorical_cols])
usable_dummy_df = pd.concat([usable_df[[v for v in REQUIRED_VARS if v not in categorical_cols]],dummy_df],axis=1)
usable_dummy_df.columns


## Scale variables
By `forecast value`

In [0]:
#usable_dummy_df 
##TODO, not currently working
scaled_df = usable_dummy_df.mul(usable_dummy_df['forecast_value'], axis=0)
#scaled_df 


# Estimate Quantiles

With understanding that GDE noted that this package doesn't work right...

Start with OLS to see if example data is OK

In [0]:
lm = smf.ols('obs_value ~ forecast_value', data=usable_df).fit()

print (lm.summary())

In [0]:
print(res.summary())